In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd


from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import HopSkipJump

import warnings
warnings.filterwarnings('ignore')

## Import data

In [ ]:
## p 269
df = pd.read_csv("datatraining.txt")
df[10:20]

In [ ]:
X = df.iloc[:, 1:6]
y = df.iloc[:, 6]

In [ ]:
np.unique(y, return_counts = True)

In [ ]:
## max-min scale
Xmin = np.min(X, axis = 0)
Xmax = np.max(X, axis = 0)

In [ ]:
## p 270
def scale(X_var):
    return (X_var - Xmin) / (Xmax - Xmin)

def unscale(X_var):
    return X_var * (Xmax - Xmin) + Xmin

In [ ]:
X_scaled = scale(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 18)

## Build training pipeline

In [ ]:
## p 271
svc      = SVC(C=1.0, kernel='rbf')
pipeline = Pipeline(steps=[('svc', svc)])

In [ ]:
param_grid = {'svc__C': np.logspace(-4, 4, 5)}
search     = GridSearchCV(estimator  = pipeline, 
                          param_grid = param_grid, 
                          iid        = False, 
                          cv         = 5)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
accuracy_test_benign = search.score(X_test, y_test)
print('Accuracy on benign test samples %0.2f'%(accuracy_test_benign * 100))

## Generate adversarial examples

In [ ]:
## p 271
classifier = SklearnClassifier(model=search.best_estimator_)
attack = HopSkipJump(classifier = classifier, 
                     targeted   = False, 
                     norm       = np.inf, 
                     max_iter   = 100, 
                     max_eval   = 100,
                     init_eval  = 100, 
                     init_size  = 100)

In [ ]:
## p 272
X_test_adv = attack.generate(X_test[:50])

In [ ]:
accuracy_test_adversarial = search.score(X_test_adv, y_test[:50])
print('Accuracy on adversarial test samples %0.2f'%(accuracy_test_adversarial * 100))

In [ ]:
X_test_adv.shape

In [ ]:
X_test_adv = unscale(pd.DataFrame(X_test_adv, columns = X_test.columns.values))
X_test_adv.head()

In [ ]:
X_test = unscale(X_test)

In [ ]:
## p 273
plt.hist(X_test[:50].Temperature)

In [ ]:
plt.hist(X_test_adv.Temperature)

In [ ]:
plt.hist(X_test.CO2)

In [ ]:
plt.hist(X_test_adv.CO2)

In [ ]:
plt.scatter(X_test.Temperature[:50], X_test_adv.Temperature[:50])

In [ ]:
plt.scatter(X_test.CO2[:50], X_test_adv.CO2[:50])

In [ ]:
plt.scatter(X_test_adv.Temperature[:50], X_test_adv.CO2[:50])

In [ ]:
plt.scatter(X_test.Temperature[:50], X_test.CO2[:50])